In [1]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import math

### Inputs

In [2]:
# Read the CSV file containing SLC data
slc = pd.read_csv('../../geospacial/slc/sorted_final_slc.csv')

### Create GeoClass Table

In [3]:
# Dropping the first column containing subbasin ID
slc = slc.drop(slc.columns[0], axis=1)

In [4]:
# Create a new dataframe with three columns for SLC number, landuse number and soil type
slc_data = pd.DataFrame(columns=['SLC', 'LuLc', 'Soil Type'])

In [5]:
# Determine the number of columns in the removed_slc dataframe
num_columns = slc.shape[1]

In [6]:
# Add numbers 1-num_columns in slc_data so each slc has a number
slc_data['SLC'] = range(1, num_columns+1)  # Numbers from 1 to the number of columns in df

In [7]:
# Populate the other two columns with land use and soil type while adding leading 0s
slc_data['LuLc'] = [int(str(x).zfill(4)[:2]) for x in slc]
slc_data['Soil Type'] = [int(str(x).zfill(4)[2:]) for x in slc]

In [8]:
# Initial numbers to fill the columns with
fill_numbers = [0, 0, 0, 1, 0, 0, 2.296, 3, 0.091, 0.493, 2.296]

In [9]:
# Create a dictionary with column names and fill numbers
extra_data = {'Main crop cropid': [fill_numbers[0]] * num_columns,
        'Second crop cropid': [fill_numbers[1]] * num_columns,
        'Crop rotation group': [fill_numbers[2]] * num_columns,
        'Vegetation type': [fill_numbers[3]] * num_columns,
        'Special class code': [fill_numbers[4]] * num_columns,
        'Tile depth': [fill_numbers[5]] * num_columns,
        'Stream depth': [fill_numbers[6]] * num_columns,
        'Number of soil layers': [fill_numbers[7]] * num_columns,
        'Soil layer depth 1': [fill_numbers[8]] * num_columns,
        'Soil layer depth 2': [fill_numbers[9]] * num_columns,
        'Soil layer depth 3': [fill_numbers[10]] * num_columns}

In [10]:
# Create the DataFrame
full_data = pd.DataFrame(extra_data)

In [11]:
# Merge the two DataFrames
final_geoclass = pd.merge(slc_data, full_data, left_index=True, right_index=True)

In [12]:
final_geoclass

,SLC,LuLc,Soil Type,Main crop cropid,Second crop cropid,Crop rotation group,Vegetation type,Special class code,Tile depth,Stream depth,Number of soil layers,Soil layer depth 1,Soil layer depth 2,Soil layer depth 3
0,1,1,1,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
1,2,1,2,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
2,3,1,3,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
3,4,1,4,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
4,5,1,6,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,113,22,13,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
113,114,22,14,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
114,115,22,16,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
115,116,23,10,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296


### Special Considerations

In [13]:
# add special class and vegetation to ilake slcs: Lu 18, St 00 adjusted
ilake_mask = ((final_geoclass['LuLc'] == 12)) & ((final_geoclass['Soil Type'] == 1))
final_geoclass.loc[ilake_mask, 'Special class code'] = 2
final_geoclass.loc[ilake_mask, 'Vegetation type'] = 3
final_geoclass.loc[ilake_mask, 'Number of soil layers'] = 1 # set soil layer and depth to 1 for water
final_geoclass.loc[ilake_mask, 'Soil layer depth 1']= 1
final_geoclass.loc[ilake_mask, 'Soil layer depth 2']= 1
final_geoclass.loc[ilake_mask, 'Soil layer depth 3']= 1

In [14]:
# add vegetation code for forest
forest_mask = final_geoclass['LuLc'].isin([1, 2, 3, 13, 14, 15])
final_geoclass.loc[forest_mask, 'Vegetation type'] = 2

In [15]:
# StMary water class
water_mask = ((final_geoclass['LuLc'] == 22)) & ((final_geoclass['Soil Type'] == 10))
final_geoclass.loc[water_mask, 'Vegetation type'] = 3
final_geoclass.loc[water_mask, 'Number of soil layers'] = 1 # set soil layer and depth to 1 for water
final_geoclass.loc[water_mask, 'Soil layer depth 1']= 1
final_geoclass.loc[water_mask, 'Soil layer depth 2']= 1
final_geoclass.loc[water_mask, 'Soil layer depth 3']= 1

In [16]:
# add wetland mask
wetland_mask= final_geoclass['LuLc'].isin([8])
final_geoclass.loc[wetland_mask, 'Vegetation type'] = 3
final_geoclass.loc[wetland_mask, 'Special class code'] = 13
final_geoclass.loc[wetland_mask, 'Stream depth'] = 0 # Stream depth set to 0 for special wetland class

In [17]:
# add glacier mask
#glacier_mask= ((final_geoclass['LuLc'] == 23)) & ((final_geoclass['Soil Type'] == 10))
#final_geoclass.loc[glacier_mask, 'Special class code'] = 3

### Write Output

In [18]:
# Output merged DataFrame to a .txt file
output_file = '../../model/Seperate_Cal/seperated_model/GeoClass.txt'

In [19]:
# Add commented lines
commented_lines = [
"""! Landsat NALCMS land cover  
! 1	Temperate or sub-polar needleleaf forest	RGB (0, 61, 0)
! 2	Sub-polar taiga needleleaf forest	RGB (148, 156, 112)
! 3	Tropical or sub-tropical broadleaf evergreen forest	RGB (0, 99, 0)
! 4	Tropical or sub-tropical broadleaf deciduous forest	RGB (30, 171, 5)
! 5	Temperate or sub-polar broadleaf deciduous forest	RGB (20, 140, 61)
! 6	Mixed forest	RGB (92, 117, 43)
! 7	Tropical or sub-tropical shrubland	RGB (179, 158, 43)
! 8	Temperate or sub-polar shrubland	RGB (179, 138, 51)
! 9	Tropical or sub-tropical grassland	RGB (232, 220, 94)
! 10	Temperate or sub-polar grassland	RGB (225, 207, 138)
! 11	Sub-polar or polar shrubland-lichen-moss	RGB (156, 117, 84)
! 12	Sub-polar or polar grassland-lichen-moss	RGB (186, 212, 143)
! 13	Sub-polar or polar barren-lichen-moss	RGB (64, 138, 112)
! 14	Wetland	RGB (107, 163, 138)
! 15	Cropland	RGB (230, 174, 102)
! 16	Barren lands	RGB (168, 171, 174)
! 17	Urban	RGB (220, 33, 38)
! 18	Water	RGB (76, 112, 163)
! 19	Snow and ice	RGB (255, 250, 255)
! =====================================
! Adjusted Land Classes: 
! Milk River:
! 1	Temperate or sub-polar needleleaf forest	RGB (0, 61, 0)
! 2	Temperate or sub-polar broadleaf deciduous forest	RGB (20, 140, 61)
! 3	Mixed forest	RGB (92, 117, 43)
! 4	Temperate or sub-polar shrubland	RGB (179, 138, 51)
! 5	Temperate or sub-polar grassland	RGB (225, 207, 138)
! 6	Sub-polar or polar shrubland-lichen-moss	RGB (156, 117, 84)
! 7	Sub-polar or polar grassland-lichen-moss	RGB (186, 212, 143)
! 8	Wetland	RGB (107, 163, 138)
! 9	Cropland	RGB (230, 174, 102)
! 10	Barren lands	RGB (168, 171, 174)
! 11	Urban	RGB (220, 33, 38)
! 12	Water	RGB (76, 112, 163)
! St. Mary: 
! 13	Temperate or sub-polar needleleaf forest	RGB (0, 61, 0)
! 14	Temperate or sub-polar broadleaf deciduous forest	RGB (20, 140, 61)
! 15	Mixed forest	RGB (92, 117, 43)
! 16	Temperate or sub-polar shrubland	RGB (179, 138, 51)
! 17	Temperate or sub-polar grassland	RGB (225, 207, 138)
! 18	Wetland	RGB (107, 163, 138)
! 19	Cropland	RGB (230, 174, 102)
! 20	Barren lands	RGB (168, 171, 174)
! 21	Urban	RGB (220, 33, 38)
! 22	Water	RGB (76, 112, 163)
! 23	Snow and ice	RGB (255, 250, 255)
!													
! ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------													
!	SoilGrid V1												
!		original legend (raster_value)	description										
!	1	No Data		
!	 C 	2	 clay										
!	 SIC 	3	 silty clay										
!	 SC 	4	 sandy clay										
!	 CL 	5	 clay loam										
!	 SICL 	6	 silty clay loam										
!	 SCL 	7	 sandy clay loam										
!	 L 	8	 loam										
!	 SIL 	9	 silty loam										
!	 SL 	10	 sandy loam										
!	 SI 	11	 silt										
!	 LS 	12	 loamy sand										
!	 S 	13	  sand										
! ======================
! Adjusted Soil Types: 
! Milk River:
!	1	No Data		
!	 2	 clay										
!	 3	 silty clay										
!	 4	 sandy clay										
!	 5	 loam										
!	 6	 silty loam										
!	 7	 silt										
!	 8	 loamy sand										
!	 9	 sand										
! St Mary:
! 	10	No Data		
!	 11	 clay										
!	 12	 silty clay										
!	 13	 sandy clay										
!	14	 silty loam	
!	15	 silt
!	16	 loamy sand
!	17	  sand	
! ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------	
!          SLC	LULC	SOIL TYPE	Main crop cropid	Second crop cropid	Crop rotation group	Vegetation type	Special class code	Tile depth	Stream depth	Number of soil layers	Soil layer depth 1	Soil layer depth 2	Soil layer depth 3"""
]

In [20]:
# Open the file in write mode
with open(output_file, 'w') as file:
    # Write the commented lines
    for line in commented_lines:
        file.write(line + '\n')

In [21]:
with open(output_file, 'a') as file:
        final_geoclass.to_csv(file, sep='\t', index=False, header=False, line_terminator='\n')

C:\Users\Paul Coderre\AppData\Local\Temp\ipykernel_21472\272444280.py:2: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  final_geoclass.to_csv(file, sep='\t', index=False, header=False, line_terminator='\n')
